In [1]:
import pandas as pd
import json

In [2]:
df_inq=pd.read_csv('ind_det_n_loan_det_n_inq.csv')
df_inq.head()

,inquiry_id,ind_det_credit_debts_balance,ind_det_debts_interestRate,ind_det_debts_currentBalancet,ind_det_debts_monthlyPayment,ind_det_debts_originalAmount,ind_det_expenses_monthlyAmount,ind_det_incomes_tyoe,ind_det_incomes_amount,tax_account_connected,...,is_associate_with_criminal_history,is_equity_owned_by_lender_or_cdc,is_presenting_prurient_performances,is_defaulted_on_federal_loan,is_engaged_in_political_or_lobbying_activities,is_speculative_in_nature,is_primarily_earning_from_rent_interest_or_investments,is_for_profit_subsidiary_of_nonprofit,is_pawn_shop,is_owned_or_controlled_by_native_american_tribe
0,3b686117-5f64-4b77-800f-f8659c09da61,[231200],[4.99],[836500],[53900],[3399900],[120000],['W-2'],[950000],0,...,0,0,0,0,0,0,0,0,0,0
1,885016e6-da8d-45d1-a2ac-750b3ea4195c,"[127783, 138179, 133587]","[12.67174697328973, 4.719429640457939, 4.16270...","[1555479, 5250931, 4567936, 572292]","[43208, 23030, 126887, 4336]","[2603567, 5692357, 4745809, 948549]","[24103, 27009, 17551, 23498, 13431]",['investment'],[4436612],0,...,1,1,0,0,0,0,0,0,0,0
2,8eef7d77-c284-4b09-843a-047dd7e18f40,[1882103],[7.757329560702783],[979973],[16333],[4034197],"[150099, 389851]","['freelance', 'salary', 'freelance']","[9232498, 23918772, 10092707]",0,...,0,0,0,0,0,0,0,0,0,0
3,3e352196-a827-46ad-9d03-9e84b177fa3a,"[200694, 263705, 409472]",[9.719433611338856],[7363507],[153406],[8880918],"[12427, 41407, 20096]","['bonus', 'bonus', 'bonus']","[2583183, 2754243, 3193742]",0,...,0,0,0,0,0,0,0,0,0,0
4,78e22302-3bed-4831-8643-e2f0afc25b21,"[233046, 828660]","[5.87458765026613, 5.727945671079141, 8.824245...","[3542779, 4425641, 717522, 1233064, 5251550]","[17367, 20489, 9966, 6422, 31259]","[8362108, 5317919, 5625414, 7316587, 5984415]","[201413, 76601, 115178, 101111]","['salary', 'freelance']","[19554704, 2599493]",0,...,0,1,0,0,0,0,0,0,0,0


In [12]:
df_inq.shape

(581, 51)

In [13]:
df_external_data=pd.read_csv('external_data.csv')
df_external_data.head()

,id,provider_type,provider_data,provider_status,inquiry_id,organization_id,created_at,subinquiry_id
0,df03db66-6f09-4f94-a08d-26e206c405ff,middesk_business,"{""id"": ""3bdfde32-554f-4db3-996f-20f413d48d81"",...",success,590cd986-9c72-4f56-8f87-4d7f709bac94,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-10-16 21:23:19.02008,NaN
1,4e3d53c0-ac64-4c04-b54a-21b7883070e3,crs_credit_report,"{""data"": {""models"": [{""score"": 512}]}}",success,885016e6-da8d-45d1-a2ac-750b3ea4195c,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-09-25 18:48:21.99,NaN
2,535c90f0-bc3f-4911-96bb-628e1cf42c6f,crs_credit_report,"{""data"": {""models"": [{""score"": 619}]}}",success,8eef7d77-c284-4b09-843a-047dd7e18f40,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-09-25 18:48:21.99,NaN
3,9de1c60d-c217-4922-a62a-0f9a3f8f15d8,crs_credit_report,"{""data"": {""models"": [{""score"": 481}]}}",success,3e352196-a827-46ad-9d03-9e84b177fa3a,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-09-25 18:48:21.99,NaN
4,7fd48929-d006-44ca-a67c-77574eb10c63,crs_credit_report,"{""data"": {""models"": [{""score"": 547}]}}",success,7af7fac9-4391-4d7f-8809-1e9d56a1b6af,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2025-09-25 18:48:21.99,NaN


In [14]:
def parse_provider(row):
    data = json.loads(row['provider_data'])
    if row['provider_type'] == 'crs_credit_report':
        return pd.Series({
            'crm_score': data['data']['models'][0]['score'],
            'litigations_found': 0,
            'liens_found': 0,
            'watchlist_hit': 0,
            'has_crm': 1,
            'has_middesk': 0

        })
    elif row['provider_type'] == 'middesk_business':
        # Flatten Middesk JSON
        tasks = {t['key']: t['status'] for t in data['review']['tasks']}
        addresses = data.get('addresses', [])
        return pd.Series({
            'crm_score': 0,
            'litigations_found': 1 if tasks.get('litigations') == 'failure' else 0,
            'liens_found': 1 if tasks.get('liens') == 'failure' else 0,
            'watchlist_hit': 1 if tasks.get('watchlist') == 'failure' else 0,
            'has_crm': 0,
            'has_middesk': 1
        })
    else:
        return pd.Series({
            'crm_score': 0,
            'litigations_found': 0,
            'liens_found': 0,
            'watchlist_hit': 0,
            'has_crm': 0,
            'has_middesk': 0
        })

# Apply to dataframe
features_df = df_external_data.apply(parse_provider, axis=1)

In [15]:
df_external_data = pd.concat([df_external_data, features_df], axis=1)
df_external_data.shape

(611, 14)

In [16]:
df_external_data=df_external_data[['inquiry_id','crm_score','has_crm','has_middesk','liens_found','litigations_found','watchlist_hit']]
df_external_data.head()

,inquiry_id,crm_score,has_crm,has_middesk,liens_found,litigations_found,watchlist_hit
0,590cd986-9c72-4f56-8f87-4d7f709bac94,0,0,1,0,1,0
1,885016e6-da8d-45d1-a2ac-750b3ea4195c,512,1,0,0,0,0
2,8eef7d77-c284-4b09-843a-047dd7e18f40,619,1,0,0,0,0
3,3e352196-a827-46ad-9d03-9e84b177fa3a,481,1,0,0,0,0
4,7af7fac9-4391-4d7f-8809-1e9d56a1b6af,547,1,0,0,0,0


In [7]:
df_external_data.to_csv('external_data_cleaned.csv', index=False)

In [21]:
merged_df = pd.merge(df_inq, df_external_data, on='inquiry_id', how='left')
merged_df.shape

(582, 57)

In [22]:
merged_df.to_csv('ind_det_n_loan_det_n_inq_n_external_data.csv', index=False)

In [24]:
df_inq=pd.read_csv('ind_det_n_loan_det_n_inq_n_external_data.csv')
df_inq.head()

,inquiry_id,ind_det_credit_debts_balance,ind_det_debts_interestRate,ind_det_debts_currentBalancet,ind_det_debts_monthlyPayment,ind_det_debts_originalAmount,ind_det_expenses_monthlyAmount,ind_det_incomes_tyoe,ind_det_incomes_amount,tax_account_connected,...,is_primarily_earning_from_rent_interest_or_investments,is_for_profit_subsidiary_of_nonprofit,is_pawn_shop,is_owned_or_controlled_by_native_american_tribe,crm_score,has_crm,has_middesk,liens_found,litigations_found,watchlist_hit
0,3b686117-5f64-4b77-800f-f8659c09da61,[231200],[4.99],[836500],[53900],[3399900],[120000],['W-2'],[950000],0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,885016e6-da8d-45d1-a2ac-750b3ea4195c,"[127783, 138179, 133587]","[12.67174697328973, 4.719429640457939, 4.16270...","[1555479, 5250931, 4567936, 572292]","[43208, 23030, 126887, 4336]","[2603567, 5692357, 4745809, 948549]","[24103, 27009, 17551, 23498, 13431]",['investment'],[4436612],0,...,0,0,0,0,512.0,1.0,0.0,0.0,0.0,0.0
2,8eef7d77-c284-4b09-843a-047dd7e18f40,[1882103],[7.757329560702783],[979973],[16333],[4034197],"[150099, 389851]","['freelance', 'salary', 'freelance']","[9232498, 23918772, 10092707]",0,...,0,0,0,0,619.0,1.0,0.0,0.0,0.0,0.0
3,3e352196-a827-46ad-9d03-9e84b177fa3a,"[200694, 263705, 409472]",[9.719433611338856],[7363507],[153406],[8880918],"[12427, 41407, 20096]","['bonus', 'bonus', 'bonus']","[2583183, 2754243, 3193742]",0,...,0,0,0,0,481.0,1.0,0.0,0.0,0.0,0.0
4,78e22302-3bed-4831-8643-e2f0afc25b21,"[233046, 828660]","[5.87458765026613, 5.727945671079141, 8.824245...","[3542779, 4425641, 717522, 1233064, 5251550]","[17367, 20489, 9966, 6422, 31259]","[8362108, 5317919, 5625414, 7316587, 5984415]","[201413, 76601, 115178, 101111]","['salary', 'freelance']","[19554704, 2599493]",0,...,0,0,0,0,685.0,1.0,0.0,0.0,0.0,0.0


In [25]:
df=pd.read_csv('acquisition_details.csv')
df.head()

,business_name,business_type,business_city,business_state,business_description,naics_code,ever_filed_bankruptcy,bankruptcy_description,ongoing_lawsuits,lawsuits_description,future_owners,inquiry_id,user_id,organization_id,id,meta_status,created_at,updated_at,business_address,business_zip_code
0,The Just Ok Corp,S Corporation,Austin,TX,We do things that are just ok.,541714,no,NaN,no,NaN,"[{""id"": ""0e9358b2-9b89-4b75-9781-d5fab12d758e""...",3b686117-5f64-4b77-800f-f8659c09da61,272gigmmb46o8f0g1uj8k,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,da0f0507-39c1-41c3-8e5b-ed6e72ed4554,t,2025-10-12 19:37:40.455+00,NaN,900 E 11th St,78702
1,"Harvey, Gusikowski and Kulas",LLC,Stockton,CT,Inflammatio ulciscor denuo tendo tondeo verto ...,326121,notSure,Sortitus dolores vulgaris ascit curatio. Carme...,notSure,Tabgo abundans aegre. Adsum circumvenio acies ...,"[{""id"": ""e029531d-5e1b-4881-b4a2-752e493b8c34""...",ab12748e-3165-442e-9ece-3a79dff66d1a,rrjl1noiucssh1y2i69hz,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,2c9a90c4-c05d-4f8b-b4d3-92c32c47e5f0,t,2025-09-24 18:33:50.532+00,NaN,624 Lincoln Street,43631
2,Herzog Inc,S Corporation,East Savannah,MI,Aufero tyrannus talis explicabo aliquid stultu...,112519,notSure,Acerbitas colligo patruus condico caries aperi...,no,NaN,"[{""id"": ""d864cf66-c1c6-4674-91a9-27192b7b7891""...",69d00ffe-46c7-47a9-846b-940c6e9f3aae,yn6phntq8zq60fq9tpesw,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,58171afa-3e6f-47e3-b74c-9c471771d6e3,t,2025-10-03 00:17:53.38+00,NaN,93201 Mireya View,15620
3,Murphy Inc,Joint venture,New Eulaliaburgh,HI,Aeternus cena curo censura vespillo tepesco ta...,111930,notSure,Tener ventito aut tertius caelum canto quae. A...,yes,Contigo suasoria suffoco fugit ancilla caterva...,"[{""id"": ""a5756ecc-2231-4b3c-866a-ddb4bae9f387""...",8362435c-8bf4-4cbc-8ae9-b0a8e94ae8ba,wa34hr2ejmgjfpqlrm1dj,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,849c9766-79d4-4743-adda-59cd3cb56055,t,2025-10-01 12:51:21.091+00,NaN,3109 The Crescent,32052
4,"Kuhic, Jaskolski and Leffler",C Corporation,Bayleestad,CA,Chirographum contego odio deludo deprecator fa...,221310,notSure,Validus amet aggredior demulceo thymum dolor t...,no,Succedo tum quod arcus demitto. Solitudo acced...,"[{""id"": ""f8a0c592-a4e1-4f6a-a6ce-3326e6f8a63b""...",c4596ecf-b549-4e12-a6a1-a1464c98de53,0xndtxp11f1le2rgw5f54,3ea1e598-67e0-ec66-d6ef-fdca7bdf2b19,5069cf87-6da7-4ae2-89cd-abab8ee99fac,t,2025-10-06 08:53:16.993+00,NaN,477 3rd Avenue,66705


In [26]:
df.shape

(157, 20)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157 entries, 0 to 156
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   business_name           157 non-null    object 
 1   business_type           157 non-null    object 
 2   business_city           157 non-null    object 
 3   business_state          157 non-null    object 
 4   business_description    157 non-null    object 
 5   naics_code              157 non-null    int64  
 6   ever_filed_bankruptcy   157 non-null    object 
 7   bankruptcy_description  155 non-null    object 
 8   ongoing_lawsuits        157 non-null    object 
 9   lawsuits_description    73 non-null     object 
 10  future_owners           157 non-null    object 
 11  inquiry_id              157 non-null    object 
 12  user_id                 157 non-null    object 
 13  organization_id         157 non-null    object 
 14  id                      157 non-null    ob

In [28]:
df['ongoing_lawsuits'].unique()

array(['no', 'notSure', 'yes'], dtype=object)

In [29]:
cols = ['ever_filed_bankruptcy', 'ongoing_lawsuits']
df[cols] = df[cols].replace({'no': 0, 'notSure': 1, 'yes': 2})
df[cols] = df[cols].astype(int)

/tmp/ipython-input-2712902637.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[cols] = df[cols].replace({'no': 0, 'notSure': 1, 'yes': 2})


In [30]:
df = df[['inquiry_id','business_type',
         'ever_filed_bankruptcy','ongoing_lawsuits']]

In [31]:
df.columns = [
    col if col == 'inquiry_id' else 'acquisition_' + col
    for col in df.columns
]

In [32]:
df.head()

,inquiry_id,acquisition_business_type,acquisition_ever_filed_bankruptcy,acquisition_ongoing_lawsuits
0,3b686117-5f64-4b77-800f-f8659c09da61,S Corporation,0,0
1,ab12748e-3165-442e-9ece-3a79dff66d1a,LLC,1,1
2,69d00ffe-46c7-47a9-846b-940c6e9f3aae,S Corporation,1,0
3,8362435c-8bf4-4cbc-8ae9-b0a8e94ae8ba,Joint venture,1,2
4,c4596ecf-b549-4e12-a6a1-a1464c98de53,C Corporation,1,0


In [33]:
df.to_csv('acquisition_details_cleaned.csv', index=False)

In [34]:
merged_df = pd.merge(df_inq, df, on='inquiry_id', how='inner')
merged_df['is_acquisition'] = merged_df['acquisition_business_type'].notnull().astype(int)
display(merged_df.head())

,inquiry_id,ind_det_credit_debts_balance,ind_det_debts_interestRate,ind_det_debts_currentBalancet,ind_det_debts_monthlyPayment,ind_det_debts_originalAmount,ind_det_expenses_monthlyAmount,ind_det_incomes_tyoe,ind_det_incomes_amount,tax_account_connected,...,crm_score,has_crm,has_middesk,liens_found,litigations_found,watchlist_hit,acquisition_business_type,acquisition_ever_filed_bankruptcy,acquisition_ongoing_lawsuits,is_acquisition
0,3b686117-5f64-4b77-800f-f8659c09da61,[231200],[4.99],[836500],[53900],[3399900],[120000],['W-2'],[950000],0,...,NaN,NaN,NaN,NaN,NaN,NaN,S Corporation,0,0,1
1,78e22302-3bed-4831-8643-e2f0afc25b21,"[233046, 828660]","[5.87458765026613, 5.727945671079141, 8.824245...","[3542779, 4425641, 717522, 1233064, 5251550]","[17367, 20489, 9966, 6422, 31259]","[8362108, 5317919, 5625414, 7316587, 5984415]","[201413, 76601, 115178, 101111]","['salary', 'freelance']","[19554704, 2599493]",0,...,685.0,1.0,0.0,0.0,0.0,0.0,Joint venture,0,0,1
2,3d91bc7b-1ac4-41bc-8ca3-fd5317d32e16,"[110252, 126390]","[3.337255535656008, 10.858613343586285, 5.8445...","[416381, 994728, 323301, 447914]","[2041, 27631, 3849, 7465]","[17062520, 1000000, 500000, 500000]","[17931, 29799, 12612, 17942, 43163, 30561]","['bonus', 'freelance']","[462681, 4532619]",0,...,792.0,1.0,0.0,0.0,0.0,0.0,C Corporation,2,2,1
3,7deced06-58ba-49b8-a25f-d8dfc2c0241c,[898976],"[6.158682257044468, 11.098064736890226, 5.5644...","[9185765, 2464280, 19470052]","[26396, 29337, 90139]","[24929396, 6781610, 30450577]","[116376, 22834]","['salary', 'freelance']","[19970669, 2817083]",0,...,800.0,1.0,0.0,0.0,0.0,0.0,Nonprofit organization,0,0,1
4,9d2ccc95-1d40-4018-935c-37f64bf66645,"[100526, 100290]","[8.736539738839848, 6.73126204368202, 19.86471...","[665528, 3741656, 128005]","[9243, 13557, 1524]","[1119745, 5000000, 200000]","[10807, 11653, 17575, 18584, 15204, 12490]",['freelance'],[2523598],0,...,469.0,1.0,0.0,0.0,0.0,0.0,Partnership,2,0,1


In [35]:
nan_counts = merged_df.isna().sum()
display(nan_counts[nan_counts > 0])

,0
crm_score,2
has_crm,2
has_middesk,2
liens_found,2
litigations_found,2
watchlist_hit,2


In [37]:
# Identify columns to fill with 0 (excluding 'acquisition_business_type')
cols_to_fill_zero = nan_counts[nan_counts > 0].index.tolist()

# Fill NaN values
merged_df['acquisition_business_type'] = merged_df['acquisition_business_type'].fillna('NA')
merged_df[cols_to_fill_zero] = merged_df[cols_to_fill_zero].fillna(0)

display(merged_df.isna().sum())

,0
inquiry_id,0
ind_det_credit_debts_balance,0
ind_det_debts_interestRate,0
ind_det_debts_currentBalancet,0
ind_det_debts_monthlyPayment,0
...,...
watchlist_hit,0
acquisition_business_type,0
acquisition_ever_filed_bankruptcy,0
acquisition_ongoing_lawsuits,0


In [38]:
merged_df[cols_to_fill_zero] = merged_df[cols_to_fill_zero].astype(int)
display(merged_df.head())

,inquiry_id,ind_det_credit_debts_balance,ind_det_debts_interestRate,ind_det_debts_currentBalancet,ind_det_debts_monthlyPayment,ind_det_debts_originalAmount,ind_det_expenses_monthlyAmount,ind_det_incomes_tyoe,ind_det_incomes_amount,tax_account_connected,...,crm_score,has_crm,has_middesk,liens_found,litigations_found,watchlist_hit,acquisition_business_type,acquisition_ever_filed_bankruptcy,acquisition_ongoing_lawsuits,is_acquisition
0,3b686117-5f64-4b77-800f-f8659c09da61,[231200],[4.99],[836500],[53900],[3399900],[120000],['W-2'],[950000],0,...,0,0,0,0,0,0,S Corporation,0,0,1
1,78e22302-3bed-4831-8643-e2f0afc25b21,"[233046, 828660]","[5.87458765026613, 5.727945671079141, 8.824245...","[3542779, 4425641, 717522, 1233064, 5251550]","[17367, 20489, 9966, 6422, 31259]","[8362108, 5317919, 5625414, 7316587, 5984415]","[201413, 76601, 115178, 101111]","['salary', 'freelance']","[19554704, 2599493]",0,...,685,1,0,0,0,0,Joint venture,0,0,1
2,3d91bc7b-1ac4-41bc-8ca3-fd5317d32e16,"[110252, 126390]","[3.337255535656008, 10.858613343586285, 5.8445...","[416381, 994728, 323301, 447914]","[2041, 27631, 3849, 7465]","[17062520, 1000000, 500000, 500000]","[17931, 29799, 12612, 17942, 43163, 30561]","['bonus', 'freelance']","[462681, 4532619]",0,...,792,1,0,0,0,0,C Corporation,2,2,1
3,7deced06-58ba-49b8-a25f-d8dfc2c0241c,[898976],"[6.158682257044468, 11.098064736890226, 5.5644...","[9185765, 2464280, 19470052]","[26396, 29337, 90139]","[24929396, 6781610, 30450577]","[116376, 22834]","['salary', 'freelance']","[19970669, 2817083]",0,...,800,1,0,0,0,0,Nonprofit organization,0,0,1
4,9d2ccc95-1d40-4018-935c-37f64bf66645,"[100526, 100290]","[8.736539738839848, 6.73126204368202, 19.86471...","[665528, 3741656, 128005]","[9243, 13557, 1524]","[1119745, 5000000, 200000]","[10807, 11653, 17575, 18584, 15204, 12490]",['freelance'],[2523598],0,...,469,1,0,0,0,0,Partnership,2,0,1


In [39]:
merged_df.to_csv('final_dataset_acquisition_externalData.csv', index=False)